In [20]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model

# Carpeta donde guardar
save_dir = "modelos"
os.makedirs(save_dir, exist_ok=True)

In [13]:
def create_transfer_model(base_model_fn, input_shape=(224,224,3), n_classes=1, dropout=0.2, trainable_layers=0):
    base = base_model_fn(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )

    # Congelamos todas las capas primero
    base.trainable = False

    # Si se especifican capas entrenables, las activamos desde el final
    if trainable_layers > 0:
        for layer in base.layers[-trainable_layers:]:
            layer.trainable = True

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(dropout)(x)
    output = Dense(n_classes, activation="sigmoid")(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


def train_model(model, train_gen, val_gen, epochs=20, class_weights=None):
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)
    ]
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        class_weight=class_weights,
        callbacks=callbacks
    )
    return history

def evaluate_model(model, generator, name="Model"):
    y_true = generator.classes
    y_pred = (model.predict(generator) > 0.5).astype("int32").ravel()

    print(f"\nResultados para {name}")
    print(classification_report(y_true, y_pred, target_names=list(generator.class_indices.keys())))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=generator.class_indices.keys(),
                yticklabels=generator.class_indices.keys())
    plt.title(f"Matriz de confusión - {name}")
    plt.ylabel("True")
    plt.xlabel("Predicted")
    plt.show()

    return cm

# Cargamos Datos

In [11]:
data_dir = "/kaggle/input/hampreprocessed/processed/train"

# Augmentación para entrenamiento
datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    shear_range=0.2,
    horizontal_flip=True, # Contempla manchas simétricas
    validation_split=0.15,  # Separación interna
)

# Generador de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

print(pd.Series(val_generator.classes).value_counts())
print(pd.Series(train_generator.classes).value_counts())

labels = train_generator.classes  

# Calculamos los pesos
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

# Lo convertimos en diccionario para Keras
class_weights = dict(enumerate(class_weights))
print(class_weights)

Found 7662 images belonging to 2 classes.
Found 1351 images belonging to 2 classes.
0    1069
1     282
Name: count, dtype: int64
0    6058
1    1604
Name: count, dtype: int64
{0: 0.6323869263783427, 1: 2.388403990024938}


# Entrenamiento de modelos con Transfer Learning

## VGG

In [14]:
vgg_model_old_weights = vgg_model.get_weights()

vgg_model = create_transfer_model(VGG16, trainable_layers= 4)
vgg_model.set_weights(vgg_model_old_weights)
history_vgg = train_model(vgg_model, train_generator, val_generator, class_weights=class_weights)

# Guardar
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
vgg_model.save(os.path.join(save_dir, f"vgg16_finetuned_{timestamp}.keras"))

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - accuracy: 0.6904 - loss: 0.5819 - val_accuracy: 0.7927 - val_loss: 0.4366 - learning_rate: 1.0000e-04
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - accuracy: 0.7338 - loss: 0.5060 - val_accuracy: 0.7121 - val_loss: 0.5310 - learning_rate: 1.0000e-04
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - accuracy: 0.7313 - loss: 0.4708 - val_accuracy: 0.7816 - val_loss: 0.4171 - learning_rate: 1.0000e-04
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 155s 3s/step - accuracy: 0.7512 - loss: 0.4620 - val_accuracy: 0.7535 - val_loss: 0.4636 - learning_rate: 1.0000e-04
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - accuracy: 0.7499 - loss: 0.4371 - val_accuracy: 0.7720 - val_loss: 0.4346 - learning_rate: 1.0000e-04
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 152s 3s/step - accuracy: 0.7693 - loss: 0.4263 - val_accuracy: 0.8157 - val_loss: 0.3658 - learning_rate: 1.0000e-04
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.7730 

## ResNet

In [ ]:
resnet_model = create_transfer_model(ResNet50, trainable_layers= 0)
train_model(resnet_model, train_generator, val_generator, class_weights=class_weights, epochs=40)

In [21]:
resnet_model_old_weights = resnet_model.get_weights()

resnet_model = create_transfer_model(ResNet50, trainable_layers= 4)
resnet_model.set_weights(resnet_model_old_weights)
history_resnet = train_model(resnet_model, train_generator, val_generator, class_weights=class_weights)

# Guardar
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
resnet_model.save(os.path.join(save_dir, f"resnet50_finetuned_{timestamp}.keras"))

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 172s 3s/step - accuracy: 0.5683 - loss: 0.7528 - val_accuracy: 0.2087 - val_loss: 0.7045 - learning_rate: 1.0000e-04
Epoch 2/20
14/60 ━━━━━━━━━━━━━━━━━━━━ 1:38 2s/step - accuracy: 0.4231 - loss: 0.7181

KeyboardInterrupt: 

## Inception

In [ ]:
inception_model = create_transfer_model(InceptionV3, trainable_layers= 0) 
train_model(inception_model, train_generator, val_generator, class_weights=class_weights, epochs=40)

In [18]:
inception_model_old_weights = inception_model.get_weights()

inception_model = create_transfer_model(InceptionV3, trainable_layers= 4) 
inception_model.set_weights(inception_model_old_weights)
history_incep = train_model(inception_model, train_generator, val_generator, class_weights=class_weights)

# Guardar
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
inception_model.save(os.path.join(save_dir, f"inceptionv3_finetuned_{timestamp}.keras"))

NameError: name 'InceptionV3' is not defined

# Evaluación

In [ ]:
evaluate_model(vgg_model, val_generator, name="VGG16")
evaluate_model(inception_model, val_generator, name="InceptionV3")
evaluate_model(resnet_model, val_generator, name="ResNet50")